In [1]:
import os
import re

import pandas as pd

# %pip install youtube-comment-downloader
from youtube_comment_downloader import YoutubeCommentDownloader, SORT_BY_RECENT

# %pip install wordcloud
import wordcloud
from wordcloud import WordCloud, STOPWORDS

# %pip install matplotlib
import matplotlib.pyplot as plt


In [5]:
YOUTUBES_VIDEO_ID = ["ptAaHyR2cFA"]

try:
    for video_id in YOUTUBES_VIDEO_ID:
        download_comments(video_id)
        generate_wordcloud(video_id)
except NameError as e:
    pass


Total Comments: 1321
cid                                   UgylzVO9UeZ37EmUghp4AaABAg
text           Link Discord MotionIme buat cari temen baru: h...
time                                          3 minggu yang lalu
author                                           @WindahBasudara
channel                                 UCoIiiHof6BJ85PLuLkuxuhw
votes                                                         52
replies                                                        5
photo          https://yt3.ggpht.com/8Uv2AxeLJ9px5PdeFTbtgC7c...
heart                                                      False
reply                                                      False
time_parsed                                    1728418679.736678
Name: 0, dtype: object
Total Comments: 2838
cid                                   UgxZnUYsWlTV8IQrriV4AaABAg
text           Link Discord MotionIme buat cari temen baru: h...
time                                           13 hari yang lalu
author                   

# Download Comments

In [3]:
def download_comments(video_id):
    downloader = YoutubeCommentDownloader()
    comments = downloader.get_comments(video_id, sort_by=SORT_BY_RECENT)

    comments = pd.DataFrame(comments)

    print(f"Total Comments: {len(comments)}")

    os.makedirs(f"data/{video_id}", exist_ok=True)

    # Save to CSV
    comments.to_feather(f"data/{video_id}/comments.feather")

    print(comments.iloc[0])
    return comments

# Word Cloud

In [4]:
def generate_wordcloud(video_id):
    if os.path.exists(f"data/{video_id}/all_comments.txt"):
        with open(f"data/{video_id}/all_comments.txt", "r") as f:
            all_comments = f.read()
    else:
        comments = pd.read_feather(f"data/{video_id}/comments.feather")
    
        all_comments = ""

        for i in range(len(comments)):
            comment = comments.iloc[i]['text'].lower()
            comment = re.sub('[^a-zA-Z]', '', comment)
            comment = re.sub('\s+', ' ', comment)  # Clean multiple spaces into single space
            all_comments += comment

        with open(f"data/{video_id}/all_comments.txt", "w") as f:
            f.write(all_comments)

    wordcloud = WordCloud(width=800, height=400, background_color="white").generate(
        all_comments
    )

    # Display the word cloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.savefig(f"data/{video_id}/wordcloud.png", bbox_inches='tight', dpi=300)
    plt.close()

# Word Frequency
